# This notebook for development purposes only

In [ ]:
import os 
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform
import cv2

rad2deg = 180 / np.pi

In [ ]:
%matplotlib inline

In [ ]:
path = '/scratch/ssd/cciw/sample_data/'
images = os.listdir(path)
#img = images[-3]
img = '1340_2016-07-11_1_GLN_3447.JPG'
#img = '1349_2016-07-06_2_GLN_3061.JPG'
#img = '1342_2016-07-07_2_GLN_3182.JPG'
#img = 'sudoku.png'

In [ ]:
f = os.path.join(path, img)

In [ ]:
im = cv2.imread(f)

In [ ]:
im.shape  # (height, width)

In [ ]:
ds = 8  # naive downsampling by factor of 8
plt.figure(figsize=(14, 10))
plt.imshow(im[::ds, ::ds])

In [ ]:
#img = im[::ds, ::ds].astype(np.uint8).copy()
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # convert to greyscale for edge detection
print(gray.shape)

In [ ]:
# indices, `DP' = data point
DP_1 = 0
DP_2 = 1
X_COORD = 0
Y_COORD = 1

In [ ]:
'''
@param threshold Accumulator threshold parameter. Only those lines are returned that get enough
       votes ( \f$>\texttt{threshold}\f$ ).
@param minLineLength Minimum line length. Line segments shorter than that are rejected.
@param maxLineGap Maximum allowed gap between points on the same line to link them.
'''
#img = im[::ds, ::ds].astype(np.uint8).copy()
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
img_h = img.shape[0]
img_w = img.shape[1]
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

#edges = cv2.Canny(gray, 100, 500, L2gradient=True)
edges = cv2.Canny(gray, 20, 400, L2gradient=True) # GLN_3447

# @param rho Distance resolution of the accumulator in pixels.
rho = 1  
# @param theta Angle resolution of the accumulator in radians.
theta = np.pi / 90  # note, with 180 getting too many overlapping lines.

lines  =  cv2.HoughLines(edges, rho, theta, threshold=100)
#linesP = cv2.HoughLinesP(edges, 1, theta, 110)
#linesP = cv2.HoughLinesP(edges, rho, theta, threshold=100, minLineLength=200, maxLineGap=100)

#print(lines.shape)  # (N - number of lines found, r - pixels, theta - radians)
#coords = []
N = 1

coords = np.zeros((N, 2, 2)).astype('int') # points, start/end, x/y
angles = np.zeros(N)
for i in range(len(lines[:N])):
    for r, theta in lines[i]:
        # theta is normal to the line wrt x-axis, so we subtract 90
        if r > 0:
            angles[i] = 90 - theta * 180 / np.pi
        else: 
            angles[i] = 270 - theta * 180 / np.pi
        print(i, '%.2f' % angles[i])
        a = np.cos(theta)
        b = np.sin(theta)
        x0, y0 = a * r, b * r
        x1 = int(x0 + img_w * -b)
        y1 = int(y0 + img_w *  a)
        x2 = int(x0 - img_w * -b)
        y2 = int(y0 - img_w *  a)
        #coords.append([[x1, y1], [x2, y2]])
        coords[i, DP_1, X_COORD] = x1
        coords[i, DP_1, Y_COORD] = y1
        coords[i, DP_2, X_COORD] = x2
        coords[i, DP_2, Y_COORD] = y2
        print(i, (x1, y1), (x2, y2))
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
plt.figure(figsize=(8,8))
plt.imshow(img)
plt.xlabel('x')
plt.ylabel('y')
plt.show()        

## Post processing (deprecated)

In [ ]:
tol = 5  # tolerance in degrees
ref = angles[0]
candidates = []
candidates.append((0, a))
similar_angles = []
for i, a in enumerate(angles[1:]):
    if a >= ref and ref + tol > a:
        similar_angles.append((i + 1, a))
    else:
        candidates.append((i + 1, a))
print(similar_angles)
print(candidates)

# Of the lines with a similar angle, keep only those that are offset
ref_coord = coords[0]

for i, a in similar_angles:
    if abs(coords[i, DP_1, X_COORD] - ref_coord[DP_1, X_COORD]) > 50:
        print('x-coords differ')
        candidates.append((i, a))
        ref_coord = coords[i]  # update ref_coord
        
    if abs(coords[i, DP_1, Y_COORD] - ref_coord[DP_1, Y_COORD]) > 50:
        print('y-coords differ')
        candidates.append((i, a))
        ref_coord = coords[i]  # update ref_coord

print(candidates)

plt.figure(figsize=(8,8))
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
for (i, _) in candidates:
    cv2.line(img, (coords[i, DP_1, X_COORD], coords[i, DP_1, Y_COORD]), 
                  (coords[i, DP_2, X_COORD], coords[i, DP_2, Y_COORD]), (0, 0, 255), 2)
plt.imshow(img)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Probabilistic HoughLines

In [ ]:
#plt.figure(figsize=(12, 10))
#plt.imshow(edges)

In [ ]:
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
img_h = img.shape[0]
img_w = img.shape[1]
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# @param canny_threshold1 Histeresis threshold 1
# @param canny_threshold2
canny_thresh1 = 20
canny_thresh2 = 400

# run the Canny edge detector on the rotated gray scale image
edges = cv2.Canny(gray, threshold1=canny_thresh1, threshold2=canny_thresh2, L2gradient=True) # GLN_3447

# @param rho Distance resolution of the accumulator (pixels).
rho = 1  

# @param theta Angle resolution of the accumulator (radians).
theta = np.pi / 90

# @param threshold Accumulator threshold, return lines with more than threshold of votes. (intersection points)
threshold = 100

# @param minLineLength Minimum line length. Line segments shorter than that are rejected. (pixels)
mLL = 300

# @param maxLineGap Maximum allowed gap between points on the same line to link them. (pixels)
mLG = 100

# run the probabilistic hough lines transform
linesP = cv2.HoughLinesP(edges, rho, theta, threshold=threshold, minLineLength=mLL, maxLineGap=mLG)

# @param lines The extremes of the detected lines if any (<N_LINES_FOUND>, 1, x_0, y_0, x_1, y_1). (pixels)
print('Found %d lines' % len(linesP))

N = 8  # top N results to draw
if linesP is not None:
    for i in range(len(linesP[:N])):
        l = linesP[i][0]
        cv2.line(img, (l[0], l[1]), (l[2], l[3]), (255,0,0), 3, cv2.LINE_AA)
plt.figure(figsize=(12, 10))
plt.imshow(img)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# line below origin, rho positive, angle less than 180
# line above origin, rho negative, angle less than 180
lines = cv2.HoughLines(edges, rho, theta, threshold=100)
rot_deg = 90 - lines[0][0][1] * rad2deg
t = rot_deg / rad2deg

In [ ]:
coordsP = np.zeros((np.minimum(N, len(linesP)), 2, 2)).astype('int') # points, start/end, x/y

for i in range(len(linesP[:N])):
    l = linesP[i][0]
    coordsP[i, DP_1, X_COORD] = l[0] # x1
    coordsP[i, DP_1, Y_COORD] = l[1] # y1
    coordsP[i, DP_2, X_COORD] = l[2] # x2
    coordsP[i, DP_2, Y_COORD] = l[3] # y2
    
R = np.array([[np.cos(-t), -np.sin(-t)],
              [np.sin(-t),  np.cos(-t)]])

coords = np.dot(coordsP, R)

correction_factor = coordsP[0, DP_1, Y_COORD] - coords[0, DP_1, Y_COORD]

for i in range(len(linesP[:N])):
    coords[i, DP_1, Y_COORD] += correction_factor
    coords[i, DP_2, Y_COORD] += correction_factor

In [ ]:
#plt.figure(figsize=(12, 10))
center = (coordsP[0, DP_1, X_COORD], coordsP[0, DP_1, Y_COORD])
imgr = transform.rotate(np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8), -rot_deg, center=center, resize=False)
imgr = (255 * imgr).astype(np.uint8)
plt.imshow(imgr[:, :])
plt.show()

In [ ]:
for i in range(len(linesP[:N])):
    cv2.line(imgr, (int(coords[i, DP_1, X_COORD]),
                    int(coords[i, DP_1, Y_COORD])),
                   (int(coords[i, DP_2, X_COORD]),
                    int(coords[i, DP_2, Y_COORD])),
             (128, 0, 255), 3, cv2.LINE_AA)
plt.figure(figsize=(12, 10))
plt.imshow(imgr)

# Find Intersection Points

In [ ]:
# first pass 
# find all the vertical and horizontal lines (dX or dY == 0)
# get intersection points
# use fact that it's a square to figure out horizontal distance.

In [ ]:
# process vertical lines
hlinesmask = dY == 0
vlinesmask = dX == 0
print(vlinesmask)

In [ ]:
print(vlines[0])
print(vlines_y1[0])
print(vlines_y2[0])

In [ ]:
print(hlines[0])
print(hlines_x1[0])
print(hlines_x2[0])

In [ ]:
X = 0

In [ ]:
Y = Y_COORD

In [ ]:
DP1 = DP_1
DP2 = DP_2

In [ ]:
coords

In [ ]:
def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return None

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

print(line_intersection((coords[0, DP1], coords[0, DP2]), 
                        (coords[1, DP1], coords[0, DP2])))

In [ ]:
corners = []
for i in range(len(coords)):
    for j in range(i + 1, len(coords)):
        intersection = line_intersection((coords[i, DP1], coords[i, DP2]), 
                                         (coords[j, DP1], coords[j, DP2]))
        # check if intersection is a valid corner
        if intersection is not None:
            cond1 = 0 < intersection[X] and intersection[X] < img_w
            cond2 = 0 < intersection[Y] and intersection[Y] < img_h
            if cond1 and cond2:
                corners.append(intersection)
corners = np.asarray(corners)
print(len(corners))
centroid = corners.mean(axis=0, keepdims=True)[0]
corner_dist = np.linalg.norm(corners - centroid, axis=1)
indices = np.argsort(corner_dist)
crop = corners[indices][:4].astype('int')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12), sharex=True)
ax1.scatter(corners[:, 0], corners[:, 1], c='r')
ax2.scatter(corners[:, 0], corners[:, 1], c='r')
ax1.scatter(centroid[0], centroid[1], c='k')
ax1.scatter(crop[:, 0], crop[:, 1], c='b')
ax1.set_xlim(0, img_w)
ax1.set_ylim(img_h, 0)
ax2.imshow(imgr)
ax1.set_aspect('equal')

In [ ]:
buffer = 10
cropped_img = imgr[crop[:, 1].min() + buffer:crop[:, 1].max() - buffer, 
                   crop[:, 0].min() + buffer:crop[:, 0].max() - buffer, :]
plt.imshow(cropped_img)

# Deprecated code

In [ ]:
x_start = 0
x_end   = 0
y_start = 0
y_end   = 0
counter = 0 # how many vertical lines crossed

for i in range(len(line_eqns)):
    
    if line_pts[i] == 0:
        
        min_pt = np.minimum(coordsP_r[i, DP_1, Y_COORD], coordsP_r[i, DP_2, Y_COORD])
        max_pt = np.maximum(coordsP_r[i, DP_1, Y_COORD], coordsP_r[i, DP_2, Y_COORD])
        
        plt.vlines(int(coordsP_r[i, DP_2, X_COORD]), min_pt, max_pt)
        
        print(i, int(coordsP_r[i, DP_2, X_COORD]))
        
        if int(coordsP_r[i, DP_2, X_COORD]) > x_start:
            x_start = int(coordsP_r[i, DP_2, X_COORD])
            print(counter, x_start)
        counter += 1            
        
    offset = np.minimum(coordsP_r[i, DP_1, X_COORD], coordsP_r[i, DP_2, X_COORD])
    plt.scatter(np.arange(int(line_pts[i])) +  offset * np.ones(int(line_pts[i])),
                line_eqns[i, :int(line_pts[i])])
plt.xlim(0, img_w)
plt.ylim(img_h, 0)

In [ ]:
def check_for_corner(coords, i, j, idx):
    
    #np.sign(coords[j, DP2, ])
    
    if int(coords[j, DP2, X]) >= int(coords[i, idx, X]) and \
       int(coords[j, DP1, X]) <= int(coords[i, idx, X]) and \
       int(coords[i, DP1, Y]) >= int(coords[j, idx, Y]) and \
       int(coords[i, DP2, Y]) <= int(coords[j, idx, Y]):
        return (int(coords[i, idx, X]), int(coords[j, idx, Y]))

    elif int(coords[j, DP2, X]) >= int(coords[i, idx, X]) and \
         int(coords[j, DP1, X]) <= int(coords[i, idx, X]) and \
         int(coords[i, DP2, Y]) >= int(coords[j, idx, Y]) and \
         int(coords[i, DP1, Y]) <= int(coords[j, idx, Y]):
        return (int(coords[i, idx, X]), int(coords[j, idx, Y]))

    elif int(coords[j, DP1, X]) >= int(coords[i, idx, X]) and \
         int(coords[j, DP2, X]) <= int(coords[i, idx, X]) and \
         int(coords[i, DP2, Y]) >= int(coords[j, idx, Y]) and \
         int(coords[i, DP1, Y]) <= int(coords[j, idx, Y]):
        return (int(coords[i, idx, X]), int(coords[j, idx, Y]))

    elif int(coords[j, DP1, X]) >= int(coords[i, idx, X]) and \
         int(coords[j, DP2, X]) <= int(coords[i, idx, X]) and \
         int(coords[i, DP1, Y]) >= int(coords[j, idx, Y]) and \
         int(coords[i, DP2, Y]) <= int(coords[j, idx, Y]):
        return (int(coords[i, idx, X]), int(coords[j, idx, Y]))

In [ ]:
x_start

In [ ]:
# equation of each line
#line_eqns = np.zeros((np.minimum(N, len(linesP)), np.maximum(img_h, img_w)) )

line_eqns = np.zeros((np.minimum(N, len(linesP)), np.maximum(img_h, img_w)) )

# length of each line
line_pts  = np.zeros( np.minimum(N, len(linesP)) )

for i in range(len(line_eqns)):
    dX = coords[i, DP_2, X_COORD] - coords[i, DP_1, X_COORD]
    if int(dX) != 0:
        dY = coords[i, DP_2, Y_COORD] - coords[i, DP_1, Y_COORD]
        
        #line_pts[i] = int(np.maximum(abs(dX), abs(dY)))
        line_pts[i] = int(abs(dX))
        
        m =  dY / dX  # slope, delta Y over delta X
        
        # point-slope formula y - y_1 = m(x - x_1) ==> y = mx - m * x_1 + y_1
        b = coords[i, DP_1, Y_COORD] - m * coords[i, DP_1, X_COORD]
        
        y = m * np.arange(line_pts[i]) + b  # y = mx + b
        line_eqns[i, :len(y)] = y
    else:
        print('Vertical line', i)
        
dY = (coords[:, DP_2, Y_COORD] - coords[:, DP_1, Y_COORD]).astype('int') # 0 means horizontal line
dY

dX = (coords[:, DP_2, X_COORD] - coords[:, DP_1, X_COORD]).astype('int')
dX        

In [ ]:
# vertical lines
vlines = coords[vlinesmask, DP_1, X].astype('int')
# start point of each vertical line
vlines_y1 = coords[vlinesmask, DP_1, Y].astype('int')
# end point of each vertical line
vlines_y2 = coords[vlinesmask, DP_2, Y].astype('int')

# horizontal lines
hlines = coords[hlinesmask, DP_1, Y].astype('int')
# start point of each horizontal line
hlines_x1 = coords[hlinesmask, DP_1, X].astype('int')
# end point of each horizontal line
hlines_x2 = coords[hlinesmask, DP_2, X].astype('int')

perf_corners = []
next_corners = []

# Find the corners
for i in range(len(vlines)):
    
    # check each known horizontal line against all vertical lines
    for j in range(len(hlines)):
        if hlines_x2[j] >= vlines[i] and hlines_x1[j] <= vlines[i]:
            perf_corners.append((vlines[i], hlines[j]))
        elif hlines_x2[j] <= vlines[i] and hlines_x1[j] >= vlines[i]:
            perf_corners.append((vlines[i], hlines[j]))
    
    # check the remaining lines, which are not horizontal or vertical lines, against vlines
    remaining_mask = ~ (hlinesmask | vlinesmask) # Bitwise or
    remaining_mask_cp = remaining_mask.copy()
    
    rem_lines = coords[remaining_mask, DP_1, Y].astype('int')
    rem_lines_x1 = coords[remaining_mask, DP_1, X].astype('int')
    rem_lines_x2 = coords[remaining_mask, DP_2, X].astype('int')
    
    for j in range(remaining_mask_cp.sum()):
        if rem_lines_x2[j] >= vlines[i] and rem_lines_x1[j] <= vlines[i]:
            next_corners.append((vlines[i], rem_lines[j]))
            remaining_mask[i] = 'False'
        elif rem_lines_x2[j] <= vlines[i] and rem_lines_x1[j] >= vlines[i]:
            next_corners.append((vlines[i], rem_lines[j]))
            remaining_mask[i] = 'False'
            
    # check the remaining lines against hlines
    rem_lines = coords[remaining_mask, DP_1, Y].astype('int')
    rem_lines_x1 = coords[remaining_mask, DP_1, X].astype('int')
    rem_lines_x2 = coords[remaining_mask, DP_2, X].astype('int')
    
    for j in range(len(hlines)):
        if rem_lines_x2[i] >= hlines[j] and rem_lines_x1[i] <= hlines[j]:
            next_corners.append((rem_lines[i], hlines[j]))
        elif rem_lines_x2[i] <= hlines[j] and rem_lines_x1[i] >= hlines[j]:
            next_corners.append((rem_lines[i], hlines[j]))
    
perf_corners = np.asarray(perf_corners)
next_corners = np.asarray(next_corners)
print(len(perf_corners))
print(perf_corners)
print(len(next_corners))
print(next_corners)